# 案例6：使用SCN对Mnist做分类

In [1]:
import sys
sys.path.append(r"D:\Rhitta_GPU")
import numpy as np
import cupy as cp
import rhitta.nn as nn
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)

### 第一步：载入数据集

In [2]:
loader=nn.MnistLoader()
train_x,number_labels=loader.load(r"D:\Rhitta_GPU\data\dataset")
labels = cp.array(onehot_encoder.fit_transform(cp.asnumpy(number_labels).reshape(-1, 1)))

train_x.shape,number_labels.shape,labels.shape

((60000, 784), (60000,), (60000, 10))

### 第二步：构造模型

In [3]:
model=nn.SCN(in_channels=1)
print(model)

Model:
Layer 1:  Conv2D(in_channels=1,out_channels=3,kernel_size=5,stride=1,padding=0) kernel : [3,5,5]  num_params：78 
Layer 2: LayerNorm()
Layer 3:  ReLU() 
Layer 4:  Pooling(in_channels=3,window_size=2,stride=2,mode=MaxPooling)   
Layer 5:  Conv2D(in_channels=3,out_channels=5,kernel_size=3,stride=1,padding=0) kernel : [5,3,3]  num_params：50 
Layer 6: LayerNorm()
Layer 7:  ReLU() 
Layer 8:  Pooling(in_channels=5,window_size=2,stride=2,mode=AveragePooling)   
Layer 9:  Conv2D(in_channels=5,out_channels=20,kernel_size=5,stride=1,padding=0) kernel : [20,5,5]  num_params：520 
Layer 10:  Linear(20,10)   num_params：210 
Total params：858


#### 构造完整计算图：输入输出节点，模型，损失

In [4]:
x=nn.to_tensor(size=(28,28))
label=nn.to_tensor(size=(1,10))
out=model(x)
predict=nn.Softmax(out)
loss=nn.CrossEntropyLoss(out,label)

### 第三步：选择并初始化优化器 

In [5]:
learning_rate = 0.01
optimizer = nn.Adam(nn.default_graph, loss, learning_rate=learning_rate)

###  第四步：开始训练

In [6]:
epochs = 20
batch_size = 4

for epoch in range(epochs):
    N = 50
    count = 0
    # 遍历样本训练
    for i in range(N):
        x.set_value(train_x[i].reshape(28,28))
        label.set_value(labels[i])
        optimizer.one_step()
        count+=1
        if count >= batch_size:
            optimizer.update()
            count=0
    # 遍历样本求准确率
    pred=[]
    for i in range(N):
        x.set_value(train_x[i].reshape(28,28))
        label.set_value(labels[i])
        predict.forward()
        pred.append(predict.value.flatten())
    temp=(cp.array(pred).argmax(axis=1) == number_labels[:N])
    accuracy=temp.sum()/N
    print("epoch:{} accuracy:{}".format(epoch+1,accuracy))
        

epoch:1 accuracy:0.18
epoch:2 accuracy:0.2
epoch:3 accuracy:0.24
epoch:4 accuracy:0.3
epoch:5 accuracy:0.36
epoch:6 accuracy:0.52
epoch:7 accuracy:0.54
epoch:8 accuracy:0.5
epoch:9 accuracy:0.52
epoch:10 accuracy:0.54
epoch:11 accuracy:0.66
epoch:12 accuracy:0.72
epoch:13 accuracy:0.76
epoch:14 accuracy:0.72
epoch:15 accuracy:0.8
epoch:16 accuracy:0.88
epoch:17 accuracy:0.86
epoch:18 accuracy:0.88
epoch:19 accuracy:0.9
epoch:20 accuracy:0.78
